<a href="https://colab.research.google.com/github/jpskycak/aihigh/blob/master/intro-to-ai/earlyAI_towerOfHanoiGeneralProblemSolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

General problem solver

Perform operations on an initial list to turn it into the desired final list

Hanoi case of n=3

Start with [[1,2,3],[],[]]
Goal is [[],[],[1,2,3]]

Valid operations: move leading digit from some list a to another list b
List[a] → list[a][1:]
List[b] → list[a][0]+list[b]

Construct a graph according to sequences of operations

Each operation given by a list [a,b]: transfer top of tower a to tower b

Generate list of states; update it with each of the possible actions (8 possible actions each round). Keep track of which actions match with which states ideally using a dictionary

---

Start by working through easiest example of n=2

Work through manually and then implement manually in code, then do fancy stuff for n=3


In [18]:
N = 6

def move(i,j,config):
  new_config = config.copy() # need an independent copy
  new_config[j] = [new_config[i][0]] + new_config[j] # put top disk on j
  new_config[i] = new_config[i][1:] # remove top disk from i
  return new_config

def possibilities(state):
  config = state['config']
  moves = state['moves']
  new_states = []
  for i in [i for i in [0,1,2] if config[i]!=[]]:
    for j in [j for j in [0,1,2]]:
      if config[j]==[]:
        new_states += [{'moves':moves+[[i,j]],'config':move(i,j,config)}]
      elif config[j][0]>config[i][0]:
        new_states += [{'moves':moves+[[i,j]],'config':move(i,j,config)}]
  return new_states

initial_config = [[n+1 for n in range(N)],[],[]]
statelist = [{'moves':[],'config':initial_config}]
target_config = [[],[],[n+1 for n in range(N)]]

counter = 0
configs_seen = [initial_config]
while target_config not in [state['config'] for state in statelist]:
  new_statelist = []
  for state in statelist:
    for possible_state in possibilities(state):
      possible_config = possible_state['config']
      if possible_config not in configs_seen:
        new_statelist += [possible_state]
        configs_seen += [possible_config]
  statelist = new_statelist.copy()
  counter += 1
  print(counter)
  
for state in statelist:
  if state['config'] == target_config:
    print(state)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
{'moves': [[0, 1], [0, 2], [1, 2], [0, 1], [2, 0], [2, 1], [0, 1], [0, 2], [1, 2], [1, 0], [2, 0], [1, 2], [0, 1], [0, 2], [1, 2], [0, 1], [2, 0], [2, 1], [0, 1], [2, 0], [1, 2], [1, 0], [2, 0], [2, 1], [0, 1], [0, 2], [1, 2], [0, 1], [2, 0], [2, 1], [0, 1], [0, 2], [1, 2], [1, 0], [2, 0], [1, 2], [0, 1], [0, 2], [1, 2], [1, 0], [2, 0], [2, 1], [0, 1], [2, 0], [1, 2], [1, 0], [2, 0], [1, 2], [0, 1], [0, 2], [1, 2], [0, 1], [2, 0], [2, 1], [0, 1], [0, 2], [1, 2], [1, 0], [2, 0], [1, 2], [0, 1], [0, 2], [1, 2]], 'config': [[], [], [1, 2, 3, 4, 5, 6]]}
